<a href="https://www.kaggle.com/code/gudalarushi/lid-xlmr?scriptVersionId=238145876" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessi

/kaggle/input/lid-dataset/LID_train.csv


In [2]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you 

In [3]:
import ast
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer, 
    EarlyStoppingCallback
)

from datasets import Dataset


2025-05-06 11:23:36.841477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746530617.039974      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746530617.096020      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Load CSV
Lid_train = pd.read_csv('/kaggle/input/lid-dataset/LID_train.csv')

# Extract majority label from annotation list
def extract_majority_language(tag_string):
    try:
        tag_list = ast.literal_eval(tag_string)
        lang_labels = [item['value'] for item in tag_list]
        most_common_lang = Counter(lang_labels).most_common(1)[0][0]
        return most_common_lang
    except:
        return "unk"

Lid_train['majority_lang'] = Lid_train['Predicted Tags'].apply(extract_majority_language)

# Encode labels
le = LabelEncoder()
Lid_train['label'] = le.fit_transform(Lid_train['majority_lang'])


In [5]:
train_df, temp_df = train_test_split(Lid_train, test_size=0.2, stratify=Lid_train['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)


In [6]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [7]:
train_dataset = Dataset.from_pandas(train_df[['Sentences', 'label']].rename(columns={'Sentences': 'text'}))
val_dataset = Dataset.from_pandas(val_df[['Sentences', 'label']].rename(columns={'Sentences': 'text'}))
test_dataset = Dataset.from_pandas(test_df[['Sentences', 'label']].rename(columns={'Sentences': 'text'}))

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


Map:   0%|          | 0/14834 [00:00<?, ? examples/s]

Map:   0%|          | 0/1854 [00:00<?, ? examples/s]

Map:   0%|          | 0/1855 [00:00<?, ? examples/s]

In [8]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}


In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(le.classes_)
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
training_args = TrainingArguments(
    output_dir='/kaggle/working/xlmr_results',
    learning_rate=2e-5,
    eval_strategy="epoch",
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir=None,  # ⛔️ No logs
    report_to="none",  # ⛔️ Don't use TensorBoard
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.325300,0.264320,0.905070,0.796488,0.727894,0.755984
2,0.218800,0.197126,0.920712,0.810573,0.783063,0.794339
3,0.175700,0.239714,0.919633,0.818254,0.804041,0.810921
4,0.143900,0.489517,0.890507,0.796649,0.751802,0.762894
5,0.105200,0.424329,0.911543,0.771771,0.819964,0.785268


TrainOutput(global_step=4640, training_loss=0.18681467709870175, metrics={'train_runtime': 1151.8846, 'train_samples_per_second': 257.561, 'train_steps_per_second': 16.113, 'total_flos': 4878780548221440.0, 'train_loss': 0.18681467709870175, 'epoch': 5.0})

In [12]:
trainer.evaluate(eval_dataset=test_dataset)


{'eval_loss': 0.2033727616071701,
 'eval_accuracy': 0.9169811320754717,
 'eval_precision': 0.8887938624686508,
 'eval_recall': 0.713179920399563,
 'eval_f1': 0.7515725875242647,
 'eval_runtime': 6.3915,
 'eval_samples_per_second': 290.23,
 'eval_steps_per_second': 18.149,
 'epoch': 5.0}